In [4]:
from tensorflow.keras.models import load_model, clone_model
import numpy as np

In [20]:
model_paths = ["model/client_model{}.h5".format(i) for i in range(10)]
models = [load_model(model_path) for model_path in model_paths]

In [3]:
model1=load_model("model/client_model5.h5")
model2=load_model("model/client_model3.h5")
model3=load_model("model/client_model9.h5")
model4=load_model("model/client_model4.h5")
model5=load_model("model/client_model0.h5")

In [ ]:
len(model1.get_weights())

In [ ]:
model1.layers

In [ ]:
len(model1.layers)

In [ ]:
model1.layers[1].get_weights()

In [ ]:
model1.get_weights()

In [ ]:
model2.get_weights()

In [ ]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

In [ ]:
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(3840,19)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['Accuracy', 'Precision', 'Recall','AUC'])
    return model

In [5]:
combined_model=clone_model(model1)

In [21]:
# combined_model=cnnmodel()
combined_model.set_weights([np.mean([model.get_weights()[i] for model in models], axis=0) for i in range(len(model1.get_weights()))])
combined_model.compile('adam',loss='binary_crossentropy',metrics=['Accuracy', 'Precision', 'Recall','AUC'])

In [23]:
len(combined_model.get_weights())

16

In [24]:
combined_model.get_weights()

[array([[[ 7.79938698e-02, -2.65273750e-02, -9.04584676e-03,
           6.66300207e-03, -5.82135431e-02],
         [ 1.21552922e-01, -1.29078791e-01, -2.84799747e-02,
          -2.94777448e-03,  9.11851674e-02],
         [ 5.49895652e-02,  9.77844186e-03, -1.49984900e-02,
          -1.16642214e-01,  2.33803745e-02],
         [-7.66136032e-03,  6.36033416e-02, -4.09623310e-02,
           4.57595009e-03, -4.62328717e-02],
         [-9.26199555e-02,  2.72075888e-02,  5.31512126e-02,
          -5.62565029e-02, -1.71034951e-02],
         [ 6.68345317e-02, -4.69917729e-02,  7.14297369e-02,
           8.38564709e-02, -1.01723401e-02],
         [-6.96145147e-02, -5.52424788e-02, -2.90793628e-02,
           4.06393409e-03,  2.02831663e-02],
         [ 4.78165820e-02,  4.78191860e-02,  7.22788274e-03,
           4.88817766e-02, -9.53085199e-02],
         [ 3.06191202e-02, -5.43209799e-02,  1.39817297e-02,
           4.40664329e-02, -3.43128256e-02],
         [-2.24994682e-02, -7.87321292e-03,  4

In [18]:
from glob import glob
import mne
from sklearn.preprocessing import StandardScaler
import numpy as np

In [10]:
all_files_path=glob('test/*.edf')
print(len(all_files_path))
healthy_file_path=[i for i in all_files_path if  'H' in i.split('\\')[1]]
patient_file_path=[i for i in all_files_path if  'M' in i.split('\\')[1]]

print(len(healthy_file_path),len(patient_file_path))
def read_data(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    # Select a specific channel
    channel_to_keep = ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE']  
      
    # Replace with the name of the channel you want to keep
    raw.pick_channels(channel_to_keep)
    raw.set_eeg_reference()
    raw.filter(l_freq=30,h_freq=100)#1-4=delta, 4-8=theta, 8-12=alpha, 12-30=beta, 30-100=gamma
    epochs=mne.make_fixed_length_epochs(raw,duration=15,overlap=1)
    epochs=epochs.get_data()
    scaler = StandardScaler()
    data = scaler.fit_transform(epochs.reshape(-1,epochs.shape[-1])).reshape(epochs.shape)
    return data #trials,channel,length

18
9 9


In [11]:
%%capture
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]

In [12]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
print(len(data_list),len(label_list))
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
data_array=np.moveaxis(data_array,1,2)

9 9
18 18


In [25]:
combined_model.evaluate(data_array, label_array)

17/17 [==============================] - 1s 21ms/step - loss: 0.6927 - Accuracy: 0.5267 - precision: 0.5267 - recall: 1.0000 - auc: 0.5000        


[0.6927360892295837, 0.5267034769058228, 0.5267034769058228, 1.0, 0.5]